# 10 Highest EQAO Ratings of Elementary Schools in Toronto Distric Schools (TDSB) & Toronto Catholic Distric School Board (TCDSB)

In [1]:
%matplotlib notebook

In [2]:
import gmaps
import numpy as np
import pandas as pd
import requests
import time
import json
from pprint import pprint
from config import gkey

In [3]:
toronto_schools = pd.read_csv("output_data/toronto_schools.csv")
catholic_schools = pd.read_csv("output_data/toronto_catholic_schools.csv")
home_prices = pd.read_csv("output_data/toronto_api_postal_combined and filtered with price range 100K to 10M.csv")

#### Average Home Prices by FSA

In [4]:
home_prices.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
0,19,Malvern,114900,43.803925,-79.216507,#105 - 8 Silverbell Grve,Toronto,M1B 2L7,M1B
1,46,Malvern,125000,43.805328,-79.220230,#310 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
2,47,Malvern,125000,43.805328,-79.220230,#101 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
3,74,Malvern,139900,43.805328,-79.220230,#405 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
4,79,Malvern,143000,43.803925,-79.216507,#504 - 8 SILVER BELL GRVE,Toronto,M1B 2L7,M1B


In [5]:
prices_fsa = home_prices[["Price ($)", "lat", "lng", "FSA"]]
prices_fsa.head()

,Price ($),lat,lng,FSA
0,114900,43.803925,-79.216507,M1B
1,125000,43.805328,-79.220230,M1B
2,125000,43.805328,-79.220230,M1B
3,139900,43.805328,-79.220230,M1B
4,143000,43.803925,-79.216507,M1B


In [6]:
avg_prices_fsa = prices_fsa.groupby(["FSA"]).mean()
avg_prices_fsa = avg_prices_fsa.rename(columns={"Price ($)":"AVG HOME PRICES"})
avg_prices_fsa["AVG HOME PRICES"] = avg_prices_fsa["AVG HOME PRICES"].round()
avg_prices_fsa.head()

,AVG HOME PRICES,lat,lng
FSA,,,
M1B,516127.0,43.803708,-79.217831
M1C,945254.0,43.789515,-79.159737
M1E,619287.0,43.772357,-79.193029
M1G,655781.0,43.770924,-79.219387
M1H,497626.0,43.775830,-79.246489


#### 10 Highest TDSB EQAO Ratings

In [7]:
toronto_schools.sort_values("RATINGS", ascending=False).head(10)

,SCHOOL NAME,ADDRESS,CITY,POSTAL CODE,FSA,RANKING (OUT OF 3046),RATINGS
11,Fleming,20 Littles Road,Scarborough,M1B 5B5,M1B,25,9.5
166,Whitney,119 Rosedale Heights Drive,Toronto,M4T 1C7,M4T,25,9.5
245,Humber Valley Village,65 Hartfield Road,Etobicoke,M9A 3E1,M9A,34,9.3
164,John Ross Robertson,130 Glengrove Avenue West,Toronto,M4R 1P2,M4R,34,9.3
129,Denlow,50 Denlow Boulevard,North York,M3B 1P7,M3B,34,9.3
115,Beverly Glen,85 Beverly Glen Boulevard,Agincourt,M1W 1W4,M1W,43,9.2
163,Blythwood,2 Strathgowan Crescent,Toronto,M4N 2Z5,M4N,47,9.1
233,Swansea,207 Windermere Avenue,Toronto,M6S 3J9,M6S,47,9.1
211,Ossington/Old Orchard,380 Ossington Avenue,Toronto,M6J 3A5,M6J,47,9.1
150,Withrow Avenue,25 Bain Avenue,Toronto,M4K 1E5,M4K,58,9.0


In [8]:
highest_ratings_tdsb = toronto_schools[toronto_schools["RATINGS"] > 8.9]
highest_ratings_tdsb

,SCHOOL NAME,ADDRESS,CITY,POSTAL CODE,FSA,RANKING (OUT OF 3046),RATINGS
11,Fleming,20 Littles Road,Scarborough,M1B 5B5,M1B,25,9.5
88,Iroquois,265 Chartland Boulevard South,Agincourt,M1S 2S6,M1S,58,9.0
115,Beverly Glen,85 Beverly Glen Boulevard,Agincourt,M1W 1W4,M1W,43,9.2
129,Denlow,50 Denlow Boulevard,North York,M3B 1P7,M3B,34,9.3
150,Withrow Avenue,25 Bain Avenue,Toronto,M4K 1E5,M4K,58,9.0
163,Blythwood,2 Strathgowan Crescent,Toronto,M4N 2Z5,M4N,47,9.1
164,John Ross Robertson,130 Glengrove Avenue West,Toronto,M4R 1P2,M4R,34,9.3
166,Whitney,119 Rosedale Heights Drive,Toronto,M4T 1C7,M4T,25,9.5
211,Ossington/Old Orchard,380 Ossington Avenue,Toronto,M6J 3A5,M6J,47,9.1
233,Swansea,207 Windermere Avenue,Toronto,M6S 3J9,M6S,47,9.1


In [9]:
highest_tdsb = highest_ratings_tdsb[["SCHOOL NAME", "ADDRESS", "POSTAL CODE", "FSA", "RATINGS"]]
highest_tdsb

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS
11,Fleming,20 Littles Road,M1B 5B5,M1B,9.5
88,Iroquois,265 Chartland Boulevard South,M1S 2S6,M1S,9.0
115,Beverly Glen,85 Beverly Glen Boulevard,M1W 1W4,M1W,9.2
129,Denlow,50 Denlow Boulevard,M3B 1P7,M3B,9.3
150,Withrow Avenue,25 Bain Avenue,M4K 1E5,M4K,9.0
163,Blythwood,2 Strathgowan Crescent,M4N 2Z5,M4N,9.1
164,John Ross Robertson,130 Glengrove Avenue West,M4R 1P2,M4R,9.3
166,Whitney,119 Rosedale Heights Drive,M4T 1C7,M4T,9.5
211,Ossington/Old Orchard,380 Ossington Avenue,M6J 3A5,M6J,9.1
233,Swansea,207 Windermere Avenue,M6S 3J9,M6S,9.1


In [10]:
highest_tdsb_homes = pd.merge(highest_tdsb, avg_prices_fsa, on="FSA")
highest_tdsb_homes

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,lat,lng
0,Fleming,20 Littles Road,M1B 5B5,M1B,9.5,516127.0,43.803708,-79.217831
1,Iroquois,265 Chartland Boulevard South,M1S 2S6,M1S,9.0,477552.0,43.786757,-79.275211
2,Beverly Glen,85 Beverly Glen Boulevard,M1W 1W4,M1W,9.2,519808.0,43.804687,-79.319888
3,Denlow,50 Denlow Boulevard,M3B 1P7,M3B,9.3,2256431.0,43.745368,-79.355629
4,Withrow Avenue,25 Bain Avenue,M4K 1E5,M4K,9.0,1245190.0,43.678382,-79.350719
5,Blythwood,2 Strathgowan Crescent,M4N 2Z5,M4N,9.1,2080118.0,43.725359,-79.388251
6,John Ross Robertson,130 Glengrove Avenue West,M4R 1P2,M4R,9.3,783480.0,43.708232,-79.402570
7,Whitney,119 Rosedale Heights Drive,M4T 1C7,M4T,9.5,1150980.0,43.689395,-79.392303
8,Ossington/Old Orchard,380 Ossington Avenue,M6J 3A5,M6J,9.1,890931.0,43.645020,-79.419622
9,Swansea,207 Windermere Avenue,M6S 3J9,M6S,9.1,738098.0,43.643246,-79.476346


#### 10 Highest TCDSB EQAO Ratings

In [11]:
catholic_schools.sort_values("RATINGS", ascending=False).head(10)

,SCHOOL NAME,ADDRESS,CITY,POSTAL CODE,FSA,RANK (OUT OF 3046),RATINGS
104,St Sebastian,717 Brock Avenue,Toronto,M6H 3P1,M6H,1,10.0
133,St Clement,4319 Bloor Street West,Etobicoke,M9C 2A2,M9C,84,8.8
84,St Michael's Choir,66 Bond St,Toronto,M5B 1X2,M5B,84,8.8
132,Josyf Cardinal Slipyj,35 West Deane Park Drive,Toronto,M9B 2R5,M9B,84,8.8
56,St Bonaventure,1340 Leslie Street,Toronto,M3C 2K9,M3C,102,8.7
81,Our Lady of Perpetual Help,1 1/2 Garfield Avenue,Toronto,M4T 1E6,M4T,123,8.6
141,St Demetrius,125 La Rose Avenue,Toronto,M9P 1A6,M9P,163,8.4
40,St Henry,100 Bamburgh Circle,Scarborough,M1W 3R3,M1W,194,8.3
122,St Josaphat,110 Tenth Street,Toronto,M8V 3G1,M8V,194,8.3
35,Prince of Peace,255 Alton Towers Circle,Scarborough,M1V 4E7,M1V,194,8.3


In [12]:
highest_ratings_tcdsb = catholic_schools[catholic_schools["RATINGS"] > 8.2]
highest_ratings_tcdsb

,SCHOOL NAME,ADDRESS,CITY,POSTAL CODE,FSA,RANK (OUT OF 3046),RATINGS
35,Prince of Peace,255 Alton Towers Circle,Scarborough,M1V 4E7,M1V,194,8.3
40,St Henry,100 Bamburgh Circle,Scarborough,M1W 3R3,M1W,194,8.3
56,St Bonaventure,1340 Leslie Street,Toronto,M3C 2K9,M3C,102,8.7
81,Our Lady of Perpetual Help,1 1/2 Garfield Avenue,Toronto,M4T 1E6,M4T,123,8.6
84,St Michael's Choir,66 Bond St,Toronto,M5B 1X2,M5B,84,8.8
104,St Sebastian,717 Brock Avenue,Toronto,M6H 3P1,M6H,1,10.0
122,St Josaphat,110 Tenth Street,Toronto,M8V 3G1,M8V,194,8.3
132,Josyf Cardinal Slipyj,35 West Deane Park Drive,Toronto,M9B 2R5,M9B,84,8.8
133,St Clement,4319 Bloor Street West,Etobicoke,M9C 2A2,M9C,84,8.8
141,St Demetrius,125 La Rose Avenue,Toronto,M9P 1A6,M9P,163,8.4


In [13]:
highest_tcdsb = highest_ratings_tcdsb[["SCHOOL NAME", "ADDRESS", "POSTAL CODE", "FSA", "RATINGS"]]
highest_tcdsb

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS
35,Prince of Peace,255 Alton Towers Circle,M1V 4E7,M1V,8.3
40,St Henry,100 Bamburgh Circle,M1W 3R3,M1W,8.3
56,St Bonaventure,1340 Leslie Street,M3C 2K9,M3C,8.7
81,Our Lady of Perpetual Help,1 1/2 Garfield Avenue,M4T 1E6,M4T,8.6
84,St Michael's Choir,66 Bond St,M5B 1X2,M5B,8.8
104,St Sebastian,717 Brock Avenue,M6H 3P1,M6H,10.0
122,St Josaphat,110 Tenth Street,M8V 3G1,M8V,8.3
132,Josyf Cardinal Slipyj,35 West Deane Park Drive,M9B 2R5,M9B,8.8
133,St Clement,4319 Bloor Street West,M9C 2A2,M9C,8.8
141,St Demetrius,125 La Rose Avenue,M9P 1A6,M9P,8.4


In [14]:
highest_tcdsb_homes = pd.merge(highest_tcdsb, avg_prices_fsa, on="FSA")
highest_tcdsb_homes

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,lat,lng
0,Prince of Peace,255 Alton Towers Circle,M1V 4E7,M1V,8.3,583646.0,43.817485,-79.279777
1,St Henry,100 Bamburgh Circle,M1W 3R3,M1W,8.3,519808.0,43.804687,-79.319888
2,St Bonaventure,1340 Leslie Street,M3C 2K9,M3C,8.7,645781.0,43.722653,-79.333331
3,Our Lady of Perpetual Help,1 1/2 Garfield Avenue,M4T 1E6,M4T,8.6,1150980.0,43.689395,-79.392303
4,St Michael's Choir,66 Bond St,M5B 1X2,M5B,8.8,496430.0,43.658569,-79.379649
5,St Sebastian,717 Brock Avenue,M6H 3P1,M6H,10.0,990385.0,43.664873,-79.439509
6,St Josaphat,110 Tenth Street,M8V 3G1,M8V,8.3,543356.0,43.621381,-79.484469
7,Josyf Cardinal Slipyj,35 West Deane Park Drive,M9B 2R5,M9B,8.8,731393.0,43.645064,-79.546506
8,St Clement,4319 Bloor Street West,M9C 2A2,M9C,8.8,460065.0,43.633744,-79.568397
9,St Demetrius,125 La Rose Avenue,M9P 1A6,M9P,8.4,812340.0,43.692650,-79.530180


### 10 Highest EQAO Rating for TDSB & TCDSB with Home Prices

In [15]:
highest_schools = pd.concat([highest_tdsb_homes, highest_tcdsb_homes])
highest_schools

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,lat,lng
0,Fleming,20 Littles Road,M1B 5B5,M1B,9.5,516127.0,43.803708,-79.217831
1,Iroquois,265 Chartland Boulevard South,M1S 2S6,M1S,9.0,477552.0,43.786757,-79.275211
2,Beverly Glen,85 Beverly Glen Boulevard,M1W 1W4,M1W,9.2,519808.0,43.804687,-79.319888
3,Denlow,50 Denlow Boulevard,M3B 1P7,M3B,9.3,2256431.0,43.745368,-79.355629
4,Withrow Avenue,25 Bain Avenue,M4K 1E5,M4K,9.0,1245190.0,43.678382,-79.350719
5,Blythwood,2 Strathgowan Crescent,M4N 2Z5,M4N,9.1,2080118.0,43.725359,-79.388251
6,John Ross Robertson,130 Glengrove Avenue West,M4R 1P2,M4R,9.3,783480.0,43.708232,-79.402570
7,Whitney,119 Rosedale Heights Drive,M4T 1C7,M4T,9.5,1150980.0,43.689395,-79.392303
8,Ossington/Old Orchard,380 Ossington Avenue,M6J 3A5,M6J,9.1,890931.0,43.645020,-79.419622
9,Swansea,207 Windermere Avenue,M6S 3J9,M6S,9.1,738098.0,43.643246,-79.476346


In [16]:
highest_schools = highest_schools.rename(columns={"lat":"HOME_LAT", "lng":"HOME_LNG"})
highest_schools["SCHOOL_LAT"] = ""
highest_schools["SCHOOL_LNG"] = ""
highest_schools.head()

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,HOME_LAT,HOME_LNG,SCHOOL_LAT,SCHOOL_LNG
0,Fleming,20 Littles Road,M1B 5B5,M1B,9.5,516127.0,43.803708,-79.217831,,
1,Iroquois,265 Chartland Boulevard South,M1S 2S6,M1S,9.0,477552.0,43.786757,-79.275211,,
2,Beverly Glen,85 Beverly Glen Boulevard,M1W 1W4,M1W,9.2,519808.0,43.804687,-79.319888,,
3,Denlow,50 Denlow Boulevard,M3B 1P7,M3B,9.3,2256431.0,43.745368,-79.355629,,
4,Withrow Avenue,25 Bain Avenue,M4K 1E5,M4K,9.0,1245190.0,43.678382,-79.350719,,


In [17]:
params = {"key": gkey}

for index, row in highest_schools.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    address = row["ADDRESS"]

    params['address'] = f"{address},+Toronto,+ON"

    schools_lat_lng = requests.get(base_url, params=params)
    
    schools_lat_lng = schools_lat_lng.json()

    highest_schools.loc[index, "SCHOOL_LAT"] = schools_lat_lng["results"][0]["geometry"]["location"]["lat"]
    highest_schools.loc[index, "SCHOOL_LNG"] = schools_lat_lng["results"][0]["geometry"]["location"]["lng"]

highest_schools

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,HOME_LAT,HOME_LNG,SCHOOL_LAT,SCHOOL_LNG
0,Fleming,20 Littles Road,M1B 5B5,M1B,9.5,516127.0,43.803708,-79.217831,43.8254,-79.2789
1,Iroquois,265 Chartland Boulevard South,M1S 2S6,M1S,9.0,477552.0,43.786757,-79.275211,43.8138,-79.328
2,Beverly Glen,85 Beverly Glen Boulevard,M1W 1W4,M1W,9.2,519808.0,43.804687,-79.319888,43.7329,-79.3559
3,Denlow,50 Denlow Boulevard,M3B 1P7,M3B,9.3,2256431.0,43.745368,-79.355629,43.6877,-79.3836
4,Withrow Avenue,25 Bain Avenue,M4K 1E5,M4K,9.0,1245190.0,43.678382,-79.350719,43.6551,-79.378
5,Blythwood,2 Strathgowan Crescent,M4N 2Z5,M4N,9.1,2080118.0,43.725359,-79.388251,43.6628,-79.4399
6,John Ross Robertson,130 Glengrove Avenue West,M4R 1P2,M4R,9.3,783480.0,43.708232,-79.402570,43.5994,-79.5091
7,Whitney,119 Rosedale Heights Drive,M4T 1C7,M4T,9.5,1150980.0,43.689395,-79.392303,43.6596,-79.5657
8,Ossington/Old Orchard,380 Ossington Avenue,M6J 3A5,M6J,9.1,890931.0,43.645020,-79.419622,43.6322,-79.5738
9,Swansea,207 Windermere Avenue,M6S 3J9,M6S,9.1,738098.0,43.643246,-79.476346,43.6847,-79.5216


In [18]:
highest_schools.to_csv("output_data/highest_schools.csv")